**Cash Flow Minimizer**

This program optimizes and minimizes financial transactions among multiple banks by calculating net balances and settling debts with the least number of transactions.

**Key Features:**

Supports any number of banks and transactions.
Computes net credit/debit for each bank based on input transactions.
Uses heaps to efficiently match debtors and creditors for minimal settlements.
Prints optimized payment instructions, simulating payment modes via a central hub.

This approach reduces transaction overhead and improves cash flow efficiency in interbank settlements.

In [1]:
import heapq
import random

class BankNetwork:
    def __init__(self, num_banks):
        """
        Initialize the bank network with the given number of banks.
        Creates an adjacency matrix 'graph' to record transactions between banks.
        Also creates a list of bank names for user-friendly display.
        """
        self.num_banks = num_banks
        self.graph = [[0] * num_banks for _ in range(num_banks)]
        self.bank_names = [f"Bank_{i+1}" for i in range(num_banks)]

    def add_transaction(self, debtor, creditor, amount):
        """
        Record a transaction where 'debtor' owes 'creditor' an 'amount'.
        This updates the graph matrix accordingly.
        """
        self.graph[debtor][creditor] += amount

    def calculate_net_balances(self):
        """
        Calculate the net balance for each bank:
        net_balance[i] = total amount owed to bank i - total amount bank i owes others.
        Positive net balance means bank is owed money (creditor),
        negative net balance means bank owes money (debtor).
        Returns the list of net balances.
        """
        net_balance = [0] * self.num_banks
        for i in range(self.num_banks):
            for j in range(self.num_banks):
                net_balance[i] += self.graph[j][i] - self.graph[i][j]
        return net_balance

    def minimize_transactions(self):
        """
        Minimize the number of transactions required to settle debts.
        Uses two heaps (priority queues): 
        - debtors heap contains banks with negative net balance
        - creditors heap contains banks with positive net balance
        Repeatedly matches the largest debtor with the largest creditor
        to settle amounts until all debts are cleared.
        Prints each minimized transaction with payment mode info.
        """
        net = self.calculate_net_balances()

        # Display initial net balances for reference
        print(f"\nInitial Net Balances: {dict(zip(self.bank_names, net))}\n")

        debtors = []   # Min-heap of (negative amount, bank index)
        creditors = [] # Min-heap of (negative amount, bank index) to simulate max heap

        # Separate debtors and creditors into heaps
        for i, amount in enumerate(net):
            if amount < 0:
                # Push debtor amount (negative) with bank index
                heapq.heappush(debtors, (amount, i))
            elif amount > 0:
                # Push creditor amount as negative to use min-heap as max-heap
                heapq.heappush(creditors, (-amount, i))

        # While both debtors and creditors remain unsettled
        while debtors and creditors:
            debt_amount, debtor = heapq.heappop(debtors)     # largest debtor (most negative)
            credit_amount, creditor = heapq.heappop(creditors) # largest creditor (most positive, stored negative)

            # Determine how much can be settled between this debtor and creditor
            settled_amount = min(-debt_amount, -credit_amount)

            # Update remaining amounts after settling this portion
            debt_remaining = debt_amount + settled_amount
            credit_remaining = credit_amount + settled_amount

            # Print the transaction details
            print(f"{self.bank_names[debtor]} pays {settled_amount} to {self.bank_names[creditor]} via World Bank [Payment Mode: {self.get_payment_mode()}]")

            # If debtor still owes money, push back into debtors heap
            if debt_remaining < 0:
                heapq.heappush(debtors, (debt_remaining, debtor))
            # If creditor is still owed money, push back into creditors heap
            if credit_remaining < 0:
                heapq.heappush(creditors, (credit_remaining, creditor))

    def get_payment_mode(self):
        """
        Randomly select a payment mode from a list of options.
        """
        return random.choice(['SWIFT', 'RTGS', 'NEFT', 'IMPS'])

def main():
    """
    Main function to interact with the user:
    - Input number of banks and transactions
    - Input each transaction's details
    - Initialize BankNetwork and process transactions
    - Call minimize_transactions to print optimized settlements
    """
    print("🏦 Welcome to the Bank Transaction Minimizer System")

    # Input number of banks with validation
    while True:
        try:
            n = int(input("Enter the Number of Banks (no limit): "))
            if n <= 0:
                print("Number of banks must be positive.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    # Input number of transactions with validation
    while True:
        try:
            t = int(input("Enter the Number of Transactions (no limit): "))
            if t < 0:
                print("Number of transactions cannot be negative.")
                continue
            break
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

    # Initialize the bank network
    network = BankNetwork(n)

    # Guide user for transaction input format
    print("\nEnter each transaction in the format:")
    print("(Debtor Bank Number) (Creditor Bank Number) (Amount > 0)")

    # Input transactions
    for _ in range(t):
        try:
            a, b, c = map(int, input().split())
            # Convert 1-based bank numbers to 0-based indices
            a -= 1
            b -= 1

            # Validate inputs and add transaction
            if 0 <= a < n and 0 <= b < n and c > 0:
                network.add_transaction(a, b, c)
            else:
                print("⚠️ Invalid transaction input. Ensure bank numbers are correct and amount > 0. Skipped.")
        except:
            print("⚠️ Error in input format. Skipped.")

    # Minimize and display transactions to settle debts
    network.minimize_transactions()

if __name__ == "__main__":
    main()


🏦 Welcome to the Bank Transaction Minimizer System


Enter the Number of Banks (no limit):  7
Enter the Number of Transactions (no limit):  10



Enter each transaction in the format:
(Debtor Bank Number) (Creditor Bank Number) (Amount > 0)


 1 2 500
 3 7 1000
 4 5 300
 6 4 190
 7 5 800
 3 6 200
 6 1 800
 4 2 1000
 7 6 200
 1 4 200



Initial Net Balances: {'Bank_1': 100, 'Bank_2': 1500, 'Bank_3': -1200, 'Bank_4': -910, 'Bank_5': 1100, 'Bank_6': -590, 'Bank_7': 0}

Bank_3 pays 1200 to Bank_2 via World Bank [Payment Mode: NEFT]
Bank_4 pays 910 to Bank_5 via World Bank [Payment Mode: NEFT]
Bank_6 pays 300 to Bank_2 via World Bank [Payment Mode: IMPS]
Bank_6 pays 190 to Bank_5 via World Bank [Payment Mode: SWIFT]
Bank_6 pays 100 to Bank_1 via World Bank [Payment Mode: NEFT]
